In [1]:
import os
import glob
import random
import pandas as pd
import tiktoken
import datetime
from collections import Counter
from IPython.display import display, Markdown


# user DF에서 History column과 Train column을 합쳐서 history로 사용
user = pd.read_csv('../../data/user.tsv', sep='\t', names=['User', 'History', 'Train', 'Question'])
user['History'] = user['History'] + user['Train']
user = user.drop(columns=['Train'])

history_news = pd.read_csv('../../data/history/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
train_news = pd.read_csv('../../data/train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
history_news = pd.concat([history_news, train_news], ignore_index=True)
history_news[['Category_New', 'SubCategory']] = history_news['Category'].str.split('|', expand=True)
history_news = history_news.drop('Category', axis=1).rename(columns={'Category_New': 'Category'})

question_news = pd.read_csv('../../data/test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'], parse_dates=['Publish'])
question_news[['Category_New', 'SubCategory']] = question_news['Category'].str.split('|', expand=True)
question_news = question_news.drop('Category', axis=1).rename(columns={'Category_New': 'Category'})


# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
question_news_sorted = question_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

In [6]:
user

,User,History,Question
0,U1,"N1,2016-12-31 17:13:57,2017-01-01 08:00:02;N1,...","N4296,2017-01-07 13:08:20,2017-01-08 01:35:11;"
1,U2,"N1,2016-12-31 17:13:57,2017-01-01 08:00:09;N1,...","N3589,2017-01-06 07:00:00,2017-01-07 18:06:12;..."
2,U3,"N1,2016-12-31 17:13:57,2017-01-01 08:00:09;N1,...","N4296,2017-01-07 13:08:20,2017-01-08 00:53:08;"
3,U4,"N2,2016-12-31 18:06:21,2017-01-01 08:00:15;N2,...","N4475,2017-01-07 18:55:37,2017-01-08 04:28:54;"
4,U5,"N3,2016-12-31 15:48:48,2017-01-01 08:00:36;N3,...","N3539,2017-01-06 22:06:33,2017-01-07 12:08:25;..."
...,...,...,...
21934,U21935,"N1412,2017-01-03 18:24:38,2017-01-04 07:39:56;...","N3890,2017-01-07 07:00:52,2017-01-08 07:43:48;..."
21935,U21936,"N1448,2017-01-03 20:12:15,2017-01-04 07:40:22;...","N4274,2017-01-07 13:51:45,2017-01-08 00:22:07;..."
21936,U21937,"N1379,2017-01-03 16:23:27,2017-01-04 07:44:48;...","N4125,2017-01-07 10:36:22,2017-01-07 20:36:47;..."
21937,U21938,"N1504,2017-01-03 21:56:31,2017-01-04 07:50:25;...","N3556,2017-01-06 18:49:28,2017-01-07 09:45:59;..."


In [2]:
def extract_news_ids_and_click_times(news_str):
    """
    목적: 
    - 뉴스 click history 문자열에서 뉴스 ID와 클릭 시간을 추출하여 각각의 리스트로 반환 (중복된 뉴스 ID는 제외)

    설명:
    - <news_str>을 세미콜론으로 분리한 후, 각 항목에서 뉴스 ID와 클릭 시간을 추출
    - 중복된 뉴스 ID는 <seen_ids> 집합을 사용하여 제외
    - 클릭 시간은 pandas의 <to_datetime> 함수를 사용하여 datetime 객체로 변환됨
    """
    news_ids = []     # 뉴스 ID를 저장할 리스트
    click_times = []  # 클릭 시간을 저장할 리스트
    seen_ids = set()  # 중복된 뉴스 ID를 추적하기 위한 집합
    entries = news_str.strip(';').split(';')  # 세미콜론으로 구분된 항목들 분리
    for entry in entries:
        if entry:
            parts = entry.split(',')  # 쉼표로 구분된 부분들 분리
            if len(parts) >= 3:
                news_id = parts[0]  # 첫 번째 부분을 뉴스 ID로 사용
                click_time_str = parts[-1]  # 마지막 부분을 클릭 시간으로 사용
                click_time = pd.to_datetime(click_time_str)  # 문자열을 datetime으로 변환
                if news_id not in seen_ids:
                    seen_ids.add(news_id)  # 중복 방지를 위해 집합에 추가
                    news_ids.append(news_id)
                    click_times.append(click_time)
    return news_ids, click_times

def find_similar_news(news_df, news_ids, click_times, used_ids, used_titles, low_freq_categories):
    """
    목적: 
    - 질문 뉴스와 유사한 뉴스 기사들 중 'low_freq_categories'에 해당하는 부정적 샘플을 찾아 반환
    - 각 질문 뉴스에 대해 최대 4개의 부정적 샘플을 선택

    변경 사항:
    - 사용자 history category 빈도를 기준으로 low_freq_categories를 필터링 조건으로 추가
    - 24시간 이내 뉴스 -> 조건 충족 안되면 더 과거의 뉴스
    """
    negative_ids = []
    negative_titles = []

    for news_id, click_time in zip(news_ids, click_times):
        time_window_start = click_time - pd.Timedelta(hours=24)

        # 1. 24시간 이내의 negative sample 후보 찾기
        candidate_news = news_df[
            (news_df['Publish'] >= time_window_start) & 
            (news_df['Publish'] <= click_time) & 
            (~news_df['News ID'].isin(used_ids)) & 
            (~news_df['Title'].isin(used_titles)) & 
            (news_df['SubCategory'].isin(low_freq_categories))
        ]

        # 제목 중복 제거
        candidate_news = candidate_news.drop_duplicates(subset='Title')

        needed = 4
        selected_news_list = []

        if len(candidate_news) >= needed:
            # 24시간 이내에서 충분히 구할 수 있는 경우
            selected_news = candidate_news.sample(n=needed, random_state=42)
            selected_news_list.append(selected_news)
        else:
            # 24시간 이내에서 부족한 경우
            selected_news_list.append(candidate_news)
            needed -= len(candidate_news)

            # 2. 더 과거에서 추가 검색 (가장 최신 순)
            extra_news = news_df[
                (news_df['Publish'] < time_window_start) &
                (~news_df['News ID'].isin(used_ids)) &
                (~news_df['Title'].isin(used_titles)) &
                (news_df['SubCategory'].isin(low_freq_categories))
            ]
            extra_news = extra_news.drop_duplicates(subset='Title')

            # time_window_start와의 시간 차이를 계산하여 가장 가까운 뉴스부터 선택
            extra_news = extra_news.assign(TimeDiff=(time_window_start - extra_news['Publish']).abs())
            extra_news = extra_news.sort_values(by='TimeDiff')

            extra_selected_news = extra_news.head(needed)
            selected_news_list.append(extra_selected_news)

        # 선택된 뉴스들 합치기
        selected_news = pd.concat(selected_news_list)

        # 최종적으로 제목 중복 제거 및 섞기
        selected_news = selected_news.drop_duplicates(subset='Title').sample(frac=1, random_state=42)

        # negative sample을 최대 4개만 사용
        selected_news = selected_news.head(4)

        similar_ids = selected_news['News ID'].tolist()
        similar_titles = selected_news['Title'].tolist()

        negative_ids.append(similar_ids)
        negative_titles.append(similar_titles)

        # 사용한 ID, Title 업데이트
        used_ids.update(similar_ids)
        used_titles.update(similar_titles)

    return negative_ids, negative_titles

def find_similar_question_news(news_df, news_ids, click_times, used_ids, used_titles):
    """
    목적: 
    - 질문 뉴스와 유사한 뉴스 기사들 중 부정적 샘플을 찾아 반환
    - 각 질문 뉴스에 대해 최대 4개의 부정적 샘플을 선택

    설명:
    - 각 질문 뉴스에 대해 클릭 시간 기준으로 24시간 이내에 발행된 뉴스 중에서 사용되지 않은 뉴스 ID와 제목을 가진 후보들을 찾음
    - 후보들 중에서 중복된 제목을 제거한 후, 필요한 수만큼 랜덤하게 샘플링
    - 필요한 수만큼의 샘플이 부족할 경우, 더 과거의 뉴스에서 추가로 샘플을 선택
    - 선택된 뉴스 ID와 제목을 리스트에 저장하고, 이후 중복 방지를 위해 used_ids와 used_titles를 업데이트
    """
    negative_ids = []  # negative sample의 뉴스 ID 리스트
    negative_titles = []  # negative sample의 뉴스 제목 리스트

    for id, click_time in zip(news_ids, click_times):
        time_window_start = click_time - pd.Timedelta(hours=24)
        candidate_news = news_df[
            (news_df['Publish'] >= time_window_start) & 
            (news_df['Publish'] <= click_time) & 
            (~news_df['News ID'].isin(used_ids)) & 
            (~news_df['Title'].isin(used_titles))
        ]

        # 제목별로 중복 제거
        candidate_news = candidate_news.drop_duplicates(subset='Title')

        # 필요한 negative sample 수
        needed = 4
        selected_news_list = []

        if len(candidate_news) >= needed:
            selected_news = candidate_news.sample(n=needed, random_state=42)  # 랜덤 sampling
            selected_news_list.append(selected_news)
        else:
            selected_news_list.append(candidate_news)
            needed -= len(candidate_news)

            # 추가로 필요한 negative sample을 더 먼 과거에서 가져오기
            extra_news = news_df[
                (news_df['Publish'] < time_window_start) &
                (~news_df['News ID'].isin(used_ids)) &
                (~news_df['Title'].isin(used_titles))
            ]
            extra_news = extra_news.drop_duplicates(subset='Title')
            extra_news = extra_news.assign(TimeDiff=(time_window_start - extra_news['Publish']).abs())
            extra_news = extra_news.sort_values(by='TimeDiff')
            extra_selected_news = extra_news.head(needed)
            selected_news_list.append(extra_selected_news)

        # 선택된 뉴스들을 하나의 DataFrame으로 결합
        selected_news = pd.concat(selected_news_list)

        # 최종적으로 제목 중복 제거 및 섞기
        selected_news = selected_news.drop_duplicates(subset='Title')
        selected_news = selected_news.sample(frac=1, random_state=42)

        # 만약 negative sample 수가 부족하면 가능한 만큼만 사용
        selected_news = selected_news.head(4)

        similar_ids = selected_news['News ID'].tolist()
        similar_titles = selected_news['Title'].tolist()

        negative_ids.append(similar_ids)
        negative_titles.append(similar_titles)

        # 사용된 ID, Title 업데이트
        used_ids.update(similar_ids)
        used_titles.update(similar_titles)

    return negative_ids, negative_titles

def save_user_file(user_data, output_folder_path, purpose):
    """
    목적: 
    - 사용자별 prompt data를 텍스트 파일로 저장

    설명:
    - 지정된 output_folder_path 경로에 기존의 .txt 파일들을 삭제
    - 각 사용자에 대해 user_id를 파일명으로 하는 .txt 파일을 생성하고, 해당 사용자 데이터를 저장
    """
    # 기존 파일들 삭제
    if os.path.exists(output_folder_path):
        for file_path in glob.glob(os.path.join(output_folder_path, "*.txt")):
            os.remove(file_path)
        print(f'[{purpose}] 기존 User Prompts 삭제')    

    os.makedirs(output_folder_path, exist_ok=True)  # 폴더 생성

    # user prompt를 각 user별 txt파일로 저장
    for user_id, content in user_data.items():
        file_path = os.path.join(output_folder_path, f"{user_id}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)

def save_metadata_file(meta_folder_path, token_counts_and_outputs, user_metadata, purpose):
    """
    목적: 
    - 메타데이터(토큰 수, 출력 수 등)를 텍스트 파일로 저장

    설명:
    - 지정된 meta_folder_path 경로에 기존의 .txt 파일들을 삭제
    - 각 사용자에 대해 토큰 수, 출력 수, 히스토리 수, 질문 수 등을 기록하여 output_metadata.txt 파일에 저장
    - 총 입력 토큰 수와 출력 토큰 수도 함께 기록
    """
    # 기존 파일들 삭제
    if os.path.exists(meta_folder_path):
        for file_path in glob.glob(os.path.join(meta_folder_path, "*.txt")):
            os.remove(file_path)    

    os.makedirs(meta_folder_path, exist_ok=True)    # 폴더 생성

    # metadata 저장 위치
    meta_file_path = os.path.join(meta_folder_path, "output_metadata.txt")

    total_input_tokens = 0  # 총 입력 토큰 수
    total_output_count = 0  # 총 출력 토큰 수

    # user별 metadata(token 수, history 수, question 수 등) 기록
    with open(meta_file_path, "w", encoding="utf-8") as meta_file:
        for user_id, token_count, output_count, num_news_ids, num_questions in sorted(token_counts_and_outputs, key=lambda x: int(x[0][1:])):
            output_line = (f"User ID: {user_id:<5} Input Tokens: {token_count:<6} Output Tokens: {output_count:<4}  "
                           f"History 수: {num_news_ids:<3}  Question 수: {num_questions}")
            total_input_tokens += token_count
            total_output_count += output_count

            meta_file.write(output_line + "\n")

        # 총 합계 기록
        total_line = f"\nTotal Input Tokens: {total_input_tokens}\nTotal Output Tokens: {total_output_count}"
        print(f"[{purpose}] {total_line}\n")
        meta_file.write(total_line + "\n")

def save_hidden_positions(meta_folder_path, hidden_positions_data):
    """
    목적: 
    - 질문의 정답 위치(인덱스)를 텍스트 파일로 저장

    설명:
    - 각 사용자에 대해 정답 위치를 기록하여 hidden_positions.txt 파일에 저장
    """
    hidden_positions_file_path = os.path.join(meta_folder_path, "hidden_positions.txt")
    with open(hidden_positions_file_path, "w", encoding="utf-8") as hidden_file:
        for user_id, positions in hidden_positions_data:
            hidden_file.write(f"{user_id:<5}: {positions}\n")

def generate_demonstrations(user_name, history_titles, history_negative_titles):
    """
    목적: 
    - Demo용 문장을 생성하여 사용자 프롬프트에 추가

    설명:
    - 사용자의 히스토리 제목과 부정적 샘플 제목을 기반으로 데모 문장을 생성
    - 실제 정답의 위치를 [MASK]로 표시하고, 후보 제목들을 무작위로 섞어 표시
    """
    demonstration_lines = []
    demonstration_lines.append(f"[News of Interest to the user]\n")

    for idx, (h_title, neg_titles) in enumerate(zip(history_titles, history_negative_titles)):
        candidates = neg_titles + [h_title]
        random.shuffle(candidates)  # 후보 제목들 섞기
        index_of_actual = candidates.index(h_title) + 1  # 실제 정답의 인덱스 (1부터 시작)

        demonstration_lines.append(f"{idx+1}) {user_name} prefers most {h_title} among the following five articles:")
        for j, candidate_title in enumerate(candidates):
            demonstration_lines.append(f"{j+1}: {candidate_title}")
        # demonstration_lines.append(f"The index number of the [MASK] is {index_of_actual}.\n")
        demonstration_lines.append(f"")

    return demonstration_lines

def create_prompts(purpose, model, user_count, max_question, max_history=300, save_folder="user_prompts", start_count=0):
    """
    목적: 
    - 전체 프로세스를 통해 사용자별 프롬프트를 생성하고, 관련 파일들을 저장

    설명:
    - 사용자 데이터를 순회하면서 히스토리 뉴스와 질문 뉴스의 ID 및 클릭 시간을 추출
    - positive 및 negative sample을 찾아 프롬프트를 생성
    - 생성된 프롬프트와 metadata를 파일로 저장
    - purpose에 따라 프롬프트의 내용이 달라지며, with_negative인 경우 부정적 샘플을 포함
    """

    now = datetime.datetime.now()
    formatted_date = now.strftime("%y%m%d")

    save_folder = f'../../prompts/[{formatted_date}-4] {save_folder}'

    # 사용자별 데이터 초기화
    user_data = {}
    token_counts_and_outputs = []
    hidden_positions_data = []

    # 히스토리의 최대 길이 설정
    max_history = int(f'-{max_history}')

    # tiktoken을 사용하여 토큰 수 계산
    encoding = tiktoken.encoding_for_model(model)

    # 사용자 목록 생성 (데이터프레임 'user'의 상위 user_count만 선택)
    selected_users = user.iloc[start_count:user_count]

    for idx, row in selected_users.iterrows():
        user_id = row['User']
        user_name = f"User #{user_id[1:]}"  # 사용자 이름 형식 지정

        # history 뉴스 ID 및 클릭 시간 추출 (중복 제거됨)
        history_str = row['History']
        history_news_ids, history_click_times = extract_news_ids_and_click_times(history_str)
        history_news_ids = history_news_ids[max_history:]
        history_click_times = history_click_times[max_history:]

        # history 뉴스 Title 추출
        history_titles = []
        for news_id in history_news_ids:
            matching_rows = history_news[history_news['News ID'] == news_id]
            if not matching_rows.empty:
                title = matching_rows.iloc[0]['Title']
                history_titles.append(title)

        # question 뉴스 ID 및 클릭 시간 추출 (중복 제거됨)
        question_str = row['Question']
        question_news_ids, question_click_times = extract_news_ids_and_click_times(question_str)

        # question 뉴스 제목 추출
        question_titles_list = []
        for news_id in question_news_ids:
            matching_rows = question_news[question_news['News ID'] == news_id]
            if not matching_rows.empty:
                title = matching_rows.iloc[0]['Title']
                question_titles_list.append(title)
                
                

        # used_ids와 used_titles 초기화 (이미 사용된 ID와 제목)
        used_ids = set(history_news_ids + question_news_ids)
        used_titles = set(history_titles + question_titles_list)
        

        user_history_categories = history_news[history_news['News ID'].isin(history_news_ids)]['SubCategory']
        category_counts = Counter(user_history_categories)
        all_categories = set(history_news['SubCategory'].unique()).union(question_news['SubCategory'].unique())
        low_freq_categories = {cat for cat in all_categories if category_counts[cat] <= 0}

        # history 뉴스에 대한 negative 샘플 찾기
        if purpose == 'with_negative':
            history_negative_ids, history_negative_titles = find_similar_news(
                history_news, history_news_ids, history_click_times, used_ids, used_titles, low_freq_categories
            )

            # 데모 생성
            demonstration_lines = generate_demonstrations(
                user_name, history_titles, history_negative_titles
            )
        else:
            demonstration_lines = []

        # question 뉴스에 대한 negative sample 찾기
        negative_question_ids, negative_question_titles = find_similar_question_news(
            question_news, question_news_ids, question_click_times, used_ids, used_titles
        )

        # 각 question에 대한 candidate title 준비
        all_candidate_titles = []
        hidden_positions = []

        for q_title, neg_titles in zip(question_titles_list, negative_question_titles):
            
            neg_titles = [title for title in neg_titles if title != q_title]    # neg_titles에서 q_title과 동일한 제목을 제거
            neg_titles = list(dict.fromkeys(neg_titles))    # neg_titles에서 제목의 중복 제거

            # negative 샘플이 4개 미만일 경우 처리  (현재는 반복을 중단)
            while len(neg_titles) < 4:
                break  # 일단 반복을 중단하고 현재 상태로 진행

            # 최대 4개의 negative 샘플과 q_title을 후보로 사용
            candidates = neg_titles[:4] + [q_title]
            # 후보에서 제목의 중복 제거
            candidates = list(dict.fromkeys(candidates))

            # 후보가 5개가 되도록 보장 (필요 시 추가 로직 구현 가능)
            if len(candidates) < 5:
                print(f'idx, question수 5개 미만')
                pass

            random.shuffle(candidates)  # 후보 섞기
            all_candidate_titles.append(candidates)
            index_of_actual = candidates.index(q_title) + 1  # 실제 정답의 인덱스 (1부터 시작)
            hidden_positions.append(index_of_actual)


        # 프롬프트 텍스트 생성
        prompt_lines = []

        # 데모 부분 추가
        prompt_lines.extend(demonstration_lines)

        # 사용자의 클릭 히스토리 추가
        if purpose == 'only_positive':
            prompt_lines.append(f"[Click History]\nThe news articles that {user_name} clicked before are as follows:")
            for i, title in enumerate(history_titles):
                prompt_lines.append(f"{i+1}. {title}")
            prompt_lines.append("\n")

        # question 부분 추가
        prompt_lines.append(f"[Questions]\nBased on {user_name}'s preferences, predict the index number of the news article that best fits the position labeled [MASK] for each question.\n")

        for i, candidates in enumerate(all_candidate_titles[:max_question]):
            # if purpose == 'with_negative':
            prompt_lines.append(f"Question {i+1}) {user_name} prefers most [MASK] among the following five articles:")
            # else:
            #     prompt_lines.append(f"Based on the articles the user clicked before, {user_name} prefers most [MASK] among the following five articles:")
            for j, candidate_title in enumerate(candidates):
                prompt_lines.append(f"{j+1}: {candidate_title}")
            prompt_lines.append("")
            # prompt_lines.append(f"Question {i+1}. The index number of the [MASK] is ?\n")
            num_questions = i+1

        prompt_lines.append(f"Please provide just the answers to each of {user_name}'s question without any explanations.")
        prompt_text = '\n'.join(prompt_lines)
        
        user_data[user_id] = prompt_text
        hidden_positions_data.append((user_id, hidden_positions))
        token_count = len(encoding.encode(prompt_text))  # 토큰 수 계산
        output_count = num_questions  # 출력 수
        num_history_titles = len(history_titles)  # 히스토리 제목 수
        token_counts_and_outputs.append((user_id, token_count, output_count, num_history_titles, num_questions))

    # prompt 및 metadata 저장
    output_folder_path = os.path.join(save_folder, purpose)
    save_user_file(user_data, output_folder_path, purpose)

    # metadata 파일 저장
    meta_folder_path = os.path.join(save_folder, purpose, 'metadata')
    save_metadata_file(meta_folder_path, token_counts_and_outputs, None, purpose)
    save_hidden_positions(meta_folder_path, hidden_positions_data)


## 실행
[create 함수]
- purpose = 어떤 목적으로 prompt를 생성할 것인지  [only_positive / with_negative]   
- model = 사용할 gpt (token 계산 용도)  [gpt-4o-mini / gpt-3.5-turbo]
- user_count = 몇 명의 user prompt를 생성할 것인지
- max_question = 최대 질문 수
- save_forder = 결과를 저장할 폴더 이름

In [3]:
create_prompts(purpose='only_positive',
               model="gpt-4o-mini",
               user_count=3000,
               max_question=15,
               max_history=60,
               save_folder="1~3000",
               start_count=0
               )

create_prompts(purpose='with_negative',
               model="gpt-4o-mini",
               user_count=3000,
               max_question=15,
               max_history=60,
               save_folder="1~3000",
               start_count=0
               )

[only_positive] 기존 User Prompts 삭제
[only_positive] 
Total Input Tokens: 2926099
Total Output Tokens: 13184

[with_negative] 기존 User Prompts 삭제
[with_negative] 
Total Input Tokens: 10670393
Total Output Tokens: 13184



In [3]:
create_prompts(purpose='only_positive',
               model="gpt-4o-mini",
               user_count=10000,
               max_question=15,
               max_history=60,
               save_folder="1~10000",
               start_count=0
               )

create_prompts(purpose='with_negative',
               model="gpt-4o-mini",
               user_count=10000,
               max_question=15,
               max_history=60,
               save_folder="1~10000",
               start_count=0
               )

[only_positive] 
Total Input Tokens: 9547652
Total Output Tokens: 42563

[with_negative] 
Total Input Tokens: 34801469
Total Output Tokens: 42563

